<a href="https://colab.research.google.com/github/Develtomas/Python_course/blob/main/DAG_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.sqlite.operators.sqlite import SqliteOperator
from datetime import datetime

import pandas as pd
import sqlite3
import requests
import io

CONN = sqlite3.connect('/content/example.db')

dag = DAG('dag', 
          start_date=datetime(2021, 1, 1),
          end_date=datetime(2021, 1, 4),
          schedule_interval='@daily',
          max_active_runs=1)

def extract_currency(date, **kwargs):
  url = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&base=EUR&symbols=USD&format=csv'
  response = requests.get(url).content
  data = pd.read_csv(io.StringIO(response.decode('utf-8')))

  kwargs['ti'].xcom_push(key='return_value', value=data['rate'].values[0])


def insert_to_db(data, table_name, conn):
  data.to_sql(table_name, conn, if_exists='append', index=False)

def extract_data(date, conn):
  url = f'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv'
  data = pd.read_csv(url,index_col=False)

  insert_to_db(data, 'data', conn)


read_csv_file_1 = PythonOperator(task_id='read_currency', 
                      python_callable=extract_currency,
                      op_kwargs={'date': '{{ ds }}'}, 
                      dag=dag)

  
read_csv_file_2 = PythonOperator(task_id='read_data', 
                      python_callable=extract_data,
                      op_kwargs={'date': '{{ ds }}', 'conn': CONN},
                      dag=dag)


join_data = SqliteOperator(
    task_id='join_data',
    sql='insert into join_data select currency, value, date, ' + 
      '{{ti.xcom_pull(key="return_value")}} as rate' + 
      ' from data ' +
      'where date LIKE "{{ds}}%";',
    dag=dag,
)

[read_csv_file_1, read_csv_file_2] >> join_data